In [ ]:
import os
import pandas as pd
import git
import numpy as np


def csv_file_adjust(folder,filename):
    filetype = filename.split(".")[-1]
    if "csv" in filetype:
        df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = \"utf-8\",sep=\"[_,\\t ;|:]\")#,sep =\"[_\\t ;|:]\",engine='python')\n",
        df.to_csv(folder+\"\\\\\"+filename.split(\".\")[0]+\".csv\")\n",
    
        df = df.iloc[:,0].str.split(\",\",expand=True)\n",
    "        # filename = filename.split(\".\")[0]+\".xlsx\"\n",
    "        # df.to_excel(folder+filename)\n",
    "        # #     if \"csv\" in filetype:\n",
    "        # # df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')\n",
    "        # # df = df.iloc[:,0].str.split(\",\",expand=True)\n",
    "\n",
    "def isnumber(x):\n",
    "    try:\n",
    "        int(x)\n",
    "        return True\n",
    "    except:\n",
    "        return False\n",
    "    \n",
    "def normalize_plate384(folder,filename):\n",
    "    '''Enter file to read plate and normalize data'''\n",
    "    print(folder+filename)\n",
    "    filetype = filename.split(\".\")[-1]\n",
    "    df = pd.DataFrame([])\n",
    "\n",
    "    \n",
    "    df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = \"utf-8\",sep=\"[_,\\t ;|:]\")#,sep =\"[_\\t ;|:]\",engine='python')\n",
    "    col_num = 30#df.shape[1]\n",
    "    index_start = 0\n",
    "    index_end = 16\n",
    "    column_index_end = 24\n",
    "    df_results_index = 0\n",
    "    for column in range(col_num):\n",
    "        print(\"column number\", column, type(column), col_num)\n",
    "        print(df.iloc[:,column])\n",
    "        try:\n",
    "            df_results_index = df[df.iloc[:,column].str.contains('A', na=False)].index#].isin(['A'])].index#\n",
    "            # column_header = column\n",
    "            column_index = column+1\n",
    "            print(\"line 25 matching?\",df_results_index)\n",
    "            break\n",
    "        except Exception as e:\n",
    "            print(e)\n",
    "            \n",
    "            pass\n",
    "        # print('checking index', df.iloc[:,column])\n",
    "    for i, index in enumerate(df_results_index):\n",
    "        # dateset_name = df.iloc[index,0]\n",
    "        new_header = df.iloc[index-1,column_index:column_index+column_index_end]\n",
    "        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col\n",
    "        df_results = df_results[df_results.applymap(isnumber)]\n",
    "        count_na = df_results.isna().sum().sum()\n",
    "        count_values = df_results.count().sum()\n",
    "\n",
    "        print(\"NA count:\", count_na, \"Values count:\", count_values, \"done\")\n",
    "        try:\n",
    "            if  count_na < 100 and count_values > 100:\n",
    "                print(\"checking 36\",df_results)\n",
    "                df_results = df_results.fillna(0)\n",
    "                df_results = df_results.astype(int)\n",
    "                df_results.columns = new_header\n",
    "                df_median_neg_controls = int(df_results.iloc[:,0:2].stack().median())\n",
    "                df_min = df_results.stack().min()\n",
    "                df_max = df_results.stack().max()\n",
    "                df_range = df_max-df_min\n",
    "                df_results_norm = (df_results - df_min)/df_range\n",
    "                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_norm\n",
    "                print(\"processed...\",i)\n",
    "            else:\n",
    "                print(\"skip data\")\n",
    "                pass\n",
    "        except Exception as e: \n",
    "            print(e)\n",
    "            pass\n",
    "\n",
    "    # df = df.iloc[:,column_index:column_index+column_index_end]\n",
    "    df.to_csv(folder+\"\\\\completed\\\\\"+filename.split(\".\")[0]+\"_normalized.\"+filename.split(\".\")[-1])\n",
    "    return(df)\n",
    "\n",
    "def zscore_plate384(folder,filename):\n",
    "    '''Enter file to read plate and normalize data'''\n",
    "    print(folder+filename)\n",
    "    filetype = \"csv\"\n",
    "    filename = filename.split(\".\")[0]\n",
    "    print(\"Filename z-score\",folder+filename, filetype)\n",
    "\n",
    "    df = pd.DataFrame([])\n",
    "    df = pd.read_csv(folder+\"\\\\completed\\\\\"+filename+\"_normalized.\"+filetype, encoding = \"utf-8\",sep=',')\n",
    "    col_num = 30#df.shape[1]\n",
    "    index_start = 0\n",
    "    index_end = 16\n",
    "    column_index_end = 24\n",
    "  \n",
    "    for column in range(col_num):\n",
    "        print(\"z-score column number\", column, type(column))\n",
    "        try:\n",
    "            df_results_index = df[df.iloc[:,column].str.contains('A', na=False,regex=False)].index\n",
    "            column_index = column+1\n",
    "            print(\"line 95 matching?\",df_results_index)\n",
    "            break\n",
    "        except Exception as e:\n",
    "            print(e)\n",
    "            pass\n",
    "    for i, index in enumerate(df_results_index):\n",
    "        new_header = df.iloc[index-1,column_index:column_index+column_index_end]\n",
    "        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col\n",
    "        count_na = df_results.isna().sum().sum()\n",
    "        count_values = df_results.count().sum()\n",
    "\n",
    "        print(\"zscore NA count:\", count_na, \"Values count:\", count_values, \"done\")\n",
    "        try:\n",
    "            if  count_na < 100 and count_values > 100:\n",
    "\n",
    "                df_results = df_results.fillna(0)\n",
    "                df_results = df_results.astype(float)\n",
    "\n",
    "                df_results.columns = new_header\n",
    "                df_min = df_results.stack().min()\n",
    "                df_max = df_results.stack().max()\n",
    "                df_range = df_max-df_min\n",
    "                df_results_norm = (df_results - df_min)/df_range\n",
    "\n",
    "                df_mean = df_results_norm.stack().mean()\n",
    "                df_std = df_results_norm.stack().std()\n",
    "                df_results_zscore = (df_results_norm - df_mean)/df_std\n",
    "                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_zscore\n",
    "                print(\"zscore processed...\",i)\n",
    "            else:\n",
    "                print(\"zscore skip data\")\n",
    "                pass\n",
    "        except Exception as e: \n",
    "            print(e)\n",
    "            pass\n",
    "    df.to_csv(folder+\"\\\\completed\\\\\"+filename.split(\".\")[0]+\"_zscore.csv\")\n",
    "\n",
    "    return(df)\n",
    "\n",
    "def index_csv(folder,filename):\n",
    "    print(folder+filename)\n",
    "    filetype = \"csv\"\n",
    "    filename = filename.split(\".\")[0]\n",
    "    print(\"Filename z-score\",folder+filename, filetype)\n",
    "\n",
    "    df = pd.DataFrame([])\n",
    "    df = pd.read_csv(folder+\"\\\\completed\\\\\"+filename+\"_normalized.\"+filetype, encoding = \"utf-8\",sep=',')\n",
    "    col_num = 30#df.shape[1]\n",
    "    index_start = 0\n",
    "    index_end = 16\n",
    "    column_index_end = 24\n",
    "  \n",
    "    for column in range(col_num):\n",
    "        print(\"z-score column number\", column, type(column))\n",
    "        try:\n",
    "            df_results_index = df[df.iloc[:,column].str.contains('A', na=False,regex=False)].index\n",
    "            column_index = column+1\n",
    "            print(\"line 95 matching?\",df_results_index)\n",
    "            break\n",
    "        except Exception as e:\n",
    "            print(e)\n",
    "            pass\n",
    "    for i, index in enumerate(df_results_index):\n",
    "        new_header = df.iloc[index-1,column_index:column_index+column_index_end]\n",
    "        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col\n",
    "        count_na = df_results.isna().sum().sum()\n",
    "        count_values = df_results.count().sum()\n",
    "\n",
    "        print(\"zscore NA count:\", count_na, \"Values count:\", count_values, \"done\")\n",
    "        try:\n",
    "            if  count_na < 100 and count_values > 100:\n",
    "                print(\"Dimensions: \",df_results.shape[0], df_results.shape[1])\n",
    "                for row in range(df_results.shape[0]):\n",
    "                    row_l = chr(ord('@')+row+1)\n",
    "                    for col in range(df_results.shape[1]):\n",
    "                        print(\"All results: \", row_l,col, df_results.iloc[row,col])\n",
    "        except Exception as e: \n",
    "            print(e)\n",
    "            pass\n",
    "\n",
    "repo = git.Repo('.', search_parent_directories=True)\n",
    "folder = repo.working_tree_dir +\"\\\\yeast-data\\\\\"\n",
    "\n",
    "plate_list = os.listdir(folder)\n",
    "print(plate_list,folder)\n",
    "\n",
    "for file in plate_list:\n",
    "    filetype = file.split(\".\")[-1]\n",
    "    if filetype == \"csv\":\n",
    "        print(\"Filename:\", file)\n",
    "        # csv_file_adjust(folder,file)\n",
    "        # normalize_plate384(folder,file)\n",
    "        # zscore_plate384(folder,file)\n",
    "        index_csv(folder,file)\n",
    "\n",
    "       \n",
    "    else:\n",
    "        print(filetype)\n",
    "        pass\n",